# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow.keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10

# Generator :
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 15  # Length of the video frames
nb_classes = 5  # Number of classes
nb_channel = 3  # Number of channels in images 3 for color(RGB) and 1 for Gray

In [5]:
# Helper function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [6]:
# Helper function to normalise the data
def normalize_data(data):
    return data/255

In [7]:
# Helper function to initialize all the batch image data and labels
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [8]:
def load_batch_images(source_path, folder_list, batch_num, batch_size, t,validation):
    
    batch_data, batch_labels = init_batch_data(batch_size)
    
     # We will also build an augumented batch data with affine transformation
    batch_data_aug, batch_labels_aug = init_batch_data(batch_size)
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, 30, 2)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        
        # Generate a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            ## image = imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping non symmetric frames
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            #Normal data
            batch_data[folder,idx] = (resized)
            
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [9]:
def generator(source_path, folder_list, batch_size, validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield load_batch_images(source_path, folder_list, batch, batch_size, t,validation)
            

        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield load_batch_images(source_path, folder_list, batch, batch_size, t,validation)


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [10]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


# Model Building :

Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [16]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed,\
                                    Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

In [17]:
# Input
input_shape=(nb_frames, nb_rows, nb_cols, nb_channel)

# <font color='1B3694'>--------------------------------------------Model Conv-3D--------------------------------------------

In [13]:
model = Sequential()

model.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2),padding='same'))

model.add(Conv3D(16, kernel_size=(3,3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.25))

model.add(Conv3D(32, kernel_size=(1,3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2),padding='same'))
model.add(Dropout(0.25))


#Flatten Layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(5, activation='softmax'))

In [14]:
optimiser = 'sgd'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 15, 120, 120, 8)   656       
_________________________________________________________________
activation (Activation)      (None, 15, 120, 120, 8)   0         
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 120, 120, 8)   32        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 7, 60, 60, 8)      0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 7, 60, 60, 8)      1736      
_________________________________________________________________
activation_1 (Activation)    (None, 7, 60, 60, 8)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 60, 60, 8)      32        
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [15]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [16]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the REducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [17]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [18]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=num_epochs,
                    verbose=1, 
                    callbacks=callbacks_list,
                    validation_data=val_generator, 
                    validation_steps=validation_steps,
                    class_weight=None,
                    workers=1,
                    shuffle=True,
                    initial_epoch=0)

Source path =  ./Project_data/train ; batch size =Epoch 1/10
 10
66/67 [============================>.] - ETA: 0s - loss: 2.0351 - categorical_accuracy: 0.3992Source path =  ./Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2019-12-2315_31_34.536150/model-00001-2.02669-0.40196-2.97879-0.36000.h5
67/67 [==============================] - 46s 688ms/step - loss: 2.0076 - categorical_accuracy: 0.4081 - val_loss: 2.9788 - val_categorical_accuracy: 0.3600
Epoch 2/10
66/67 [============================>.] - ETA: 0s - loss: 1.5806 - categorical_accuracy: 0.4848
Epoch 00002: saving model to model_init_2019-12-2315_31_34.536150/model-00002-1.57261-0.48507-1.12077-0.50000.h5
67/67 [==============================] - 15s 218ms/step - loss: 1.5726 - categorical_accuracy: 0.4851 - val_loss: 1.1208 - val_categorical_accuracy: 0.5000
Epoch 3/10
66/67 [============================>.] - ETA: 0s - loss: 1.2179 - categorical_accuracy: 0.5682
Epoch 00003: saving model to model_ini

<font color=#2A941B>As we can see Conv3D model is yeilding higher accuracy of 77% in training data and 81% in validation data with just 307,213 parameters.This is a good model .

Also the Conv-RNN Model was yeilding accuracy of 73% in training data and 74% in validation data with 303,901 parameters .

#### <font color=#900C3F> Hence we can say that here Conv3D is performing better than Conv-RNN in similar number of parameters!

The code for Conv-RNN model we build is given Below .

# <font color='1B3694'>--------------------------------------------Model Conv-RNN--------------------------------------------

In [18]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [19]:
model = Sequential()

model.add(TimeDistributed(Conv2D(8,
                                 kernel_size = (3, 3),
                                             activation='relu',
                                 padding='same'),
                          input_shape=input_shape))

model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(MaxPooling2D(pool_size = (2, 2) )))


model.add(TimeDistributed(Conv2D(8,
                                 kernel_size = (3, 3),
                                             activation='relu',
                                 padding='same'),
                          input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))

model.add(TimeDistributed(MaxPooling2D(pool_size = (2, 2) )))



model.add(TimeDistributed(Conv2D(16,
                                 kernel_size = (3,3),
                                 padding='same',
                                 activation='relu')))

model.add(TimeDistributed(MaxPooling2D(pool_size = (2, 2) )))
                                      

    
model.add(TimeDistributed(Conv2D(32,
                                 kernel_size = (1,3),
                                 padding='same',
                                 activation='relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size = (2, 2) )))

model.add(TimeDistributed(BatchNormalization()))
model.add(Dropout(0.25))


model.add(TimeDistributed(Flatten()))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))


# Using GRU as the RNN model along with softmax as our last layer.

model.add(GRU(128, return_sequences=False))
#model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax')) # using Softmax as last layer

In [20]:
optimiser = 'sgd'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 15, 120, 120, 8)   224       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 120, 120, 8)   32        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 15, 60, 60, 8)     0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 15, 60, 60, 8)     584       
_________________________________________________________________
time_distributed_4 (TimeDist (None, 15, 60, 60, 8)     32        
_________________________________________________________________
time_distributed_5 (TimeDist (None, 15, 30, 30, 8)     0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 15, 30, 30, 16)    1168      
__________

In [21]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [22]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the REducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

In [23]:
batch_size = 10
num_epochs = 10

In [24]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [25]:
model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=num_epochs,
                    verbose=1, 
                    callbacks=callbacks_list,
                    validation_data=val_generator, 
                    validation_steps=validation_steps,
                    class_weight=None,
                    shuffle=True,
                    workers=1,
                    initial_epoch=0)

Source path =  ./Project_data/train ; batch size = 10
Epoch 1/10
66/67 [============================>.] - ETA: 0s - loss: 1.5534 - categorical_accuracy: 0.3068Source path =  ./Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2019-12-2317_45_27.793992/model-00001-1.55323-0.30694-1.95701-0.28500.h5
67/67 [==============================] - 45s 671ms/step - loss: 1.5529 - categorical_accuracy: 0.3072 - val_loss: 1.9570 - val_categorical_accuracy: 0.2850
Epoch 2/10
66/67 [============================>.] - ETA: 0s - loss: 1.2773 - categorical_accuracy: 0.4874
Epoch 00002: saving model to model_init_2019-12-2317_45_27.793992/model-00002-1.28052-0.48507-1.59197-0.36500.h5
67/67 [==============================] - 14s 215ms/step - loss: 1.2805 - categorical_accuracy: 0.4851 - val_loss: 1.5920 - val_categorical_accuracy: 0.3650
Epoch 3/10
66/67 [============================>.] - ETA: 0s - loss: 1.1310 - categorical_accuracy: 0.5808
Epoch 00003: saving model to model_ini

The Conv-RNN Model was yeilding accuracy of 73% in training data and 74% in validation data with 303,901 parameters.